# Data Transformation Experiment

This notebook presents the data transformation experiment in `Learning Data Transformations with Minimal User Effort`.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

import plotly.graph_objs as go
from plotly.offline import plot, iplot, init_notebook_mode

from sacred import Experiment
from sacred.observers import MongoObserver

init_notebook_mode()

if ".." not in sys.path:
    sys.path.append("..")

Experiment enviroment are setup using Sacred. Experiments are recorded in MongoDB database.

In [2]:
import logging
from pathlib import Path
from datafc.eval import Evaluator
from datafc.utils.logging import setup_logging

setup_logging("../conf/logging.yaml")

# specify data path
data_folder = Path("../data")

# specify method

ex = Experiment("jupyter_ex", interactive=True)
ex.observers.append(MongoObserver.create())
ex.logger = logging.getLogger("mynotebook")


@ex.config
def config():
    # these configurations only affects udata method
    mapping_method = "sim"
    mapping_features = ["jaccard", "syn", "token_jaccard"]
    with_flashfill = False
    num_example = 1000
    k = 1


@ex.main
def run(
    dataset, method, mapping_method, mapping_features, with_flashfill, num_example, k
):
    evaluator = Evaluator(
        mapping_method, mapping_features, with_flashfill, num_example, k
    )
    return evaluator.run_dataset(data_folder / method / dataset, method)

Experiments are run on 5 different datasets: `museum`, `ijcai`, `sygus`, `nyc` and `prog`. Evaluation for FlashFill and UData systems can be run using this notebook.
* UData: run as python program included in datafc folder.
* FlashFill: run using Excel macro to call built-in FlashFill plugin in Excel. 

In [20]:
# Specify methods here
methods = ["ipbe"]

In [21]:
# Specify datasets here
datasets = ["prog", "nyc", "sygus", "ijcai", "museum"]

In [43]:
dataset_to_result = {}

for method in methods:
    print(f"Running {method} method")
    for dataset in datasets:
        print(f" - Dataset: {dataset}")
        dataset_to_result[dataset] = ex.run(config_updates={"dataset": dataset, "method": method}).result

Running ipbe method
 - Dataset: prog
 - Dataset: nyc
 - Dataset: sygus
 - Dataset: ijcai
 - Dataset: museum


## Detailed result of the experiment is shown below:

In [44]:
# Experiment that runs flashfill on museum data cannot be stored. Uncomment this cell to store the result in csv format.

# report = DatasetReport("museum", dataset_to_result["museum"])
# df = DatasetReport.generate_report_frame([report])
# df.to_csv("../result/flash_fill.csv")

In [45]:
from pymongo import MongoClient
from datafc.eval.report import DatasetReport, ScenarioReport
import qgrid

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

dataset_to_report = []

reviewing_methods = ["ipbe"]
reviewing_datasets = ["prog", "nyc", "sygus", "ijcai", "museum"]

client = MongoClient()
for method in reviewing_methods:
    for dataset in reviewing_datasets:
        if method == "flashfill" and dataset == "museum":
            continue
            
        dataset_result = client.sacred.runs.find_one(
            {"config.dataset": dataset, "config.method": method, "result": {"$ne": None}}, sort=[("$natural", -1)]
        )
        dataset_to_report.append(DatasetReport(dataset, dataset_result["result"]))

In [46]:
df = DatasetReport.generate_report_frame(dataset_to_report)
df

,name,num_scenarios,num_correct,macro_mean_acc,macro_top_k_acc,example_valid_p,example_valid_r,example_valid_f1,mean_running_time,s_valid_p,s_valid_r,s_valid_f1,valid_accuracy
0,prog,6,5,0.985714,0.985714,1.0,1.0,1.0,4.241667,1,1,1.0,1.0
1,nyc,5,4,0.972727,0.972727,1.0,1.0,1.0,36.072000,1,1,1.0,1.0
2,sygus,27,21,0.940586,0.940586,1.0,1.0,1.0,1.210700,1,1,1.0,1.0
3,ijcai,36,17,0.830475,0.830475,1.0,1.0,1.0,36.814198,1,1,1.0,1.0
4,museum,173,165,0.988341,0.988341,1.0,1.0,1.0,3.941545,1,1,1.0,1.0


### Detailed results for scenarios in each dataset:

You can review the detail of every scenario in one dataset as follows:

In [75]:
from datafc.eval.report import ScenarioReport
import qgrid

def show_result_df(dataset):
    df = ScenarioReport.generate_report_frame(dataset_to_report[dataset].scenario_to_report.values())
    display(df)
    
show_result_df("nyc")

,name,micro_acc,micro_top_k_acc,validation_p,validation_r,validation_f1,transformation_result,validation_result,running_time
0,address,0.010381,0.010381,0.989619,1.0,0.994783,False,TP,7.348250
1,city,1.000000,1.000000,1.000000,1.0,1.000000,True,TN,1.938999
2,lat-long,1.000000,1.000000,0.000000,1.0,0.000000,True,FP,0.334103
3,name,1.000000,1.000000,0.000000,1.0,0.000000,True,FP,0.047000
4,website,0.454545,0.454545,0.545455,1.0,0.705882,False,TP,1.050002
